In [1]:
import pandas as pd
import numpy as np
import itertools as it
import json
import re
import env
from collections import Counter
from pydataset import data
import pymysql
from sqlalchemy import text, create_engine
from env import host, user, pwd

In [2]:
pd.set_option('display.max_columns', None)

<div class="alert alert-info">
  <h1><strong>EXERCISE I</strong></h1>
</div>
# see?

<div class="alert alert-info">
  <h1><strong>mysql+pymysql</strong></h1>
</div>
# see?

### Establish a cxn to CodeUp SQL database

In [3]:
url = env.get_db_url('employees',user,pwd,host)

In [4]:
engine = create_engine(url)
query = text('SELECT * FROM employees LIMIT 5 OFFSET 50')
pd.read_sql(query,engine.connect())

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


### CXN established, however the df is not stored into a variable name.  Next, make an intentional string error in the SQL query.  What error message is thrown?

In [5]:
query = text('SELECT * FROM employe_s LIMIT 5 OFFSET 50')

In [6]:
pd.read_sql(query,engine.connect())

ProgrammingError: (pymysql.err.ProgrammingError) (1146, "Table 'employees.employe_s' doesn't exist")
[SQL: SELECT * FROM employe_s LIMIT 5 OFFSET 50]
(Background on this error at: https://sqlalche.me/e/14/f405)

### NB table 'employees.employe_s' DOES NOT EXIST in db

## Now read the employees.employees table and employees.titles table into seperate dfs

In [7]:
query = '''
SELECT *
FROM employees e
'''

In [8]:
employees = pd.read_sql(text(query),engine.connect())

In [9]:
employees.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12


### and now titles table ...

In [10]:
query = '''
SELECT *
FROM titles t
'''

In [11]:
titles = pd.read_sql(text(query),engine.connect())

In [12]:
titles.head()

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01


### How many rows and columns in each df?

In [13]:
employees.shape,titles.shape

((300024, 6), (443308, 4))

### Display summary statistics for each df...there are no numerical columns btw

In [14]:
employees.describe()

,emp_no
count,300024.000000
mean,253321.763392
std,161828.235540
min,10001.000000
25%,85006.750000
50%,249987.500000
75%,424993.250000
max,499999.000000


In [15]:
titles.describe()

,emp_no
count,443308.000000
mean,253075.034430
std,161853.292613
min,10001.000000
25%,84855.750000
50%,249847.500000
75%,424891.250000
max,499999.000000


### number of unique titles in titles df

In [16]:
titles

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01
...,...,...,...,...
443303,499997,Engineer,1987-08-30,1992-08-29
443304,499997,Senior Engineer,1992-08-29,9999-01-01
443305,499998,Senior Staff,1998-12-27,9999-01-01
443306,499998,Staff,1993-12-27,1998-12-27


In [17]:
titles.groupby('title').count()

,emp_no,from_date,to_date
title,,,
Assistant Engineer,15128,15128,15128
Engineer,115003,115003,115003
Manager,24,24,24
Senior Engineer,97750,97750,97750
Senior Staff,92853,92853,92853
Staff,107391,107391,107391
Technique Leader,15159,15159,15159


In [18]:
len(titles.groupby('title'))

7

### OLDEST date in to_date column?

In [19]:
titles[titles['to_date'] == titles['to_date'].min()]

,emp_no,title,from_date,to_date
16064,20869,Engineer,1985-02-17,1985-03-01


In [20]:
titles.sort_values(by='to_date').head(2)

,emp_no,title,from_date,to_date
16064,20869,Engineer,1985-02-17,1985-03-01
406358,474918,Engineer,1985-02-10,1985-03-11


### MOST RECENT date in to_date column?

In [21]:
titles[titles['to_date'] != titles['to_date'].max()]

,emp_no,title,from_date,to_date
3,10004,Engineer,1986-12-01,1995-12-01
6,10005,Staff,1989-09-12,1996-09-12
9,10007,Staff,1989-02-10,1996-02-11
10,10008,Assistant Engineer,1998-03-11,2000-07-31
11,10009,Assistant Engineer,1985-02-18,1990-02-18
...,...,...,...,...
443297,499992,Staff,1987-05-10,1992-05-09
443299,499994,Engineer,1993-02-22,1993-10-27
443301,499996,Engineer,1996-05-13,2002-05-13
443303,499997,Engineer,1987-08-30,1992-08-29


In [22]:
titles[titles['to_date'] != titles['to_date'].max()].sort_values(by='to_date',ascending = False).head(40)

,emp_no,title,from_date,to_date
154568,204324,Technique Leader,1985-02-10,2002-08-01
119478,90666,Engineer,1997-08-01,2002-08-01
387242,461960,Staff,1997-08-01,2002-08-01
319569,416185,Staff,1999-10-28,2002-08-01
67905,55876,Staff,1995-08-01,2002-08-01
399465,470224,Senior Staff,1999-08-08,2002-08-01
251244,269877,Senior Staff,1997-11-04,2002-08-01
209437,241571,Senior Engineer,1990-01-05,2002-08-01
209212,241410,Senior Engineer,2001-04-20,2002-08-01
203061,237207,Engineer,1993-08-01,2002-08-01


<div class="alert alert-warning">
  <h1><strong>EXERCISE II</strong></h1>
</div>

### Copy 'users' and 'roles' DFs from lesson

In [23]:
users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [24]:
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


### What is the result of .merge by using a Right Join?

### No Jane and No Mike from left tables

In [25]:
users.merge(roles, left_on='role_id', right_on='id', how='right', indicator=True)

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1,admin,both
1,2.0,joe,2.0,2,author,both
2,3.0,sally,3.0,3,reviewer,both
3,4.0,adam,3.0,3,reviewer,both
4,NaN,NaN,NaN,4,commenter,right_only


### What is the result of .merge by using a OUTER Join?

### All rows from all tables will show

In [26]:
users.merge(roles, left_on='role_id', right_on='id', how='outer', indicator=True)

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1.0,admin,both
1,2.0,joe,2.0,2.0,author,both
2,3.0,sally,3.0,3.0,reviewer,both
3,4.0,adam,3.0,3.0,reviewer,both
4,5.0,jane,NaN,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,NaN,left_only
6,NaN,NaN,NaN,4.0,commenter,right_only


### Drop the foreign keys from each table and try to merge them?

### A UNION of two tables one above the other

In [27]:
users.merge(roles, how='outer', indicator=True)

,id,name,role_id,_merge
0,1,bob,1.0,left_only
1,2,joe,2.0,left_only
2,3,sally,3.0,left_only
3,4,adam,3.0,left_only
4,5,jane,NaN,left_only
5,6,mike,NaN,left_only
6,1,admin,NaN,right_only
7,2,author,NaN,right_only
8,3,reviewer,NaN,right_only
9,4,commenter,NaN,right_only


In [3]:
mpg = data('mpg')

In [4]:
mpg.describe()

,displ,year,cyl,cty,hwy
count,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171
std,1.291959,4.509646,1.611534,4.255946,5.954643
min,1.600000,1999.000000,4.000000,9.000000,12.000000
25%,2.400000,1999.000000,4.000000,14.000000,18.000000
50%,3.300000,2003.500000,6.000000,17.000000,24.000000
75%,4.600000,2008.000000,8.000000,19.000000,27.000000
max,7.000000,2008.000000,8.000000,35.000000,44.000000


### Output and read documentation for 'mpg' dataset.

In [5]:
mpg=data('mpg',show_doc=True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




In [6]:
mpg = data('mpg')

### How many rows and columns in the dataset?

In [7]:
mpg.describe()

,displ,year,cyl,cty,hwy
count,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171
std,1.291959,4.509646,1.611534,4.255946,5.954643
min,1.600000,1999.000000,4.000000,9.000000,12.000000
25%,2.400000,1999.000000,4.000000,14.000000,18.000000
50%,3.300000,2003.500000,6.000000,17.000000,24.000000
75%,4.600000,2008.000000,8.000000,19.000000,27.000000
max,7.000000,2008.000000,8.000000,35.000000,44.000000


### Look at column names and make changes where needed...for example, look at the use of 'reserved words'

In [8]:
mpg.columns

Index(['manufacturer', 'model', 'displ', 'year', 'cyl', 'trans', 'drv', 'cty',
       'hwy', 'fl', 'class'],
      dtype='object')

In [9]:
mpg = mpg.rename(columns={'class':'auto_class'})
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,auto_class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


In [10]:
mpg.shape

(234, 11)

### How many different manufacturers are there?

In [11]:
mpg['manufacturer'].nunique()

15

### How many different models are there?

In [12]:
mpg['model'].nunique()

38

### mileage diff column

In [13]:
mpg['mileage_diff'] = mpg['hwy'] - mpg['cty']

In [14]:
mpg[mpg['mileage_diff'] > 0]

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,auto_class,mileage_diff
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10
...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8


### avg_mileage column

In [15]:
mpg['avg_mileage'] = (mpg['hwy'] + mpg['cty']) / 2

In [16]:
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,auto_class,mileage_diff,avg_mileage
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,23.5
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25.0
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21.0
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22.0


### is_automatic column

In [17]:
mpg['is_automatic'] = np.where(mpg['trans'].str.startswith('a'),1,0)

In [18]:
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,auto_class,mileage_diff,avg_mileage,is_automatic
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5,1
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0,0
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5,0
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5,1
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,23.5,1
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25.0,0
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21.0,1
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22.0,0


### manufacturers ranked by avg_mpg

In [19]:
mpg.groupby('manufacturer').avg_mileage.mean().sort_values(ascending=False)

manufacturer
honda         28.500000
volkswagen    25.074074
hyundai       22.750000
subaru        22.428571
audi          22.027778
toyota        21.720588
pontiac       21.700000
nissan        21.346154
chevrolet     18.447368
ford          16.680000
mercury       15.625000
jeep          15.562500
dodge         15.540541
lincoln       14.166667
land rover    14.000000
Name: avg_mileage, dtype: float64

### which transmission type has higher fuel efficiency, manual or automatic?

In [22]:
mpg.groupby('is_automatic').avg_mileage.mean().sort_values(ascending=False)

is_automatic
0    22.227273
1    19.130573
Name: avg_mileage, dtype: float64